In [7]:
import numpy as np
import pandas as pd
from helper_functions import read_photos, embeddings_to_file, get_batches
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Input
from keras.models import Model
import h5py
import config

In [2]:
dbscan = DBSCAN(eps = 480, min_samples=2, metric="euclidean")

In [3]:
#Dirs with photos
TRAIN_DIR = config.TRAIN_DIR
#TEST_DIR = "D:/test_photos/"

#CSV files with photos ids and corresponding business ids 
#test_photo_to_biz = pd.read_csv("D:/test_photo_to_biz.csv")
train_photo_to_biz = config.train_photo_to_biz

#CSV file with labels corresponding to business
lbs_pd = config.labels

In [4]:
ResNet_model = ResNet50(weights='imagenet', include_top = False)
input1 = Input(shape=(224, 224, 3), name='image_input')
features = ResNet_model(input1)
features = Flatten()(features)
features_model = Model(inputs=input1, outputs=features)

W0831 12:43:40.308590  9180 deprecation_wrapper.py:119] From C:\Users\vaitovich\AppData\Local\Continuum\anaconda3\envs\data-science\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 12:43:40.359821  9180 deprecation_wrapper.py:119] From C:\Users\vaitovich\AppData\Local\Continuum\anaconda3\envs\data-science\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 12:43:40.379079  9180 deprecation_wrapper.py:119] From C:\Users\vaitovich\AppData\Local\Continuum\anaconda3\envs\data-science\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0831 12:43:40.500344  9180 deprecation_wrapper.py:119] From C:\Users\vaitovich\AppData\Local\Continuum\anaconda3\envs\data-science\lib\site-packages\ker

In [8]:
batches, lbs = get_batches(train_photo_to_biz, lbs_pd)

In [ ]:
embeddings_to_file("embeddings", TRAIN_DIR, batches, features_model, 2000, 0)

In [ ]:
f = h5py.File("embeddings.h5", "r")
datasets = sorted([i for i in f], key=lambda x:int(x[5:]))
embeddings = f[datasets[0]][:]
for i in range(1,len(datasets)):
    embeddings = np.concatenate((embeddings, f[datasets[i]][:]), axis=0)
f.close()

In [ ]:
emb_dists = euclidean_distances(embeddings, embeddings)
sorted_emb_dists = np.sort(emb_dists)
dists_to_plot = sorted([np.sum(vec[:2])/2 for vec in sorted_emb_dists])
plt.plot(dists_to_plot)

In [ ]:
Path = "D:/result_pca.h5"
file = h5py.File(Path, "r")
datasets = sorted([i for i in file], key=lambda x:int(x[5:]))
data100 = file[datasets[0]][:]
for i in range(1,len(datasets)):
    data100 = np.concatenate([data100, file[datasets[i]][:]], axis=0)
file.close()

In [ ]:
dbscan.fit(data100)
dbscan_lbs = dbscan.labels_

In [ ]:
f = h5py.File("dbscan_labels.h5", "w")
f.create_dataset("lbs", data=dbscan_lbs)
f.close()

In [ ]:
# photos = []
# photos_id = []
# photos_to_plot = []
# for i in range(len(batches)):
#         photos_to_plot+=read_photos(batches[i][1], True)
#         photos_id+=list(batches[i][1])